In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_absolute_error
import seaborn as sns
from sklearn.model_selection import KFold
from scipy.fftpack import fft,ifft,idct
import gc

In [2]:
label=pd.read_csv('ID_Gender_Age_datasets.csv')
label=label.drop('Unnamed: 0',axis=1)

In [4]:
# without ManualExtractionData/Android data
ID_list=list(label['ID'])
file_name=['AutomaticExtractionData_IMUZCenter','ManualExtractionData/IMUZCenter','ManualExtractionData/IMUZRight',
          'ManualExtractionData/IMUZLeft']
action_name=['_Walk_1','_Walk_2','_SlopeDown','_SlopeUp']

In [5]:
# generate series new features
# including Ax,Ay,Az,Gx,Gy,Gz and AccSumAx, AccSumGx
# and the 
def generate_read_table(fileName,actionName,ID,label): 
    try:
        dataset_name= fileName + actionName
        path_name= fileName +'/'+ str(label[label['ID']==ID][dataset_name].values[0])
        if fileName=='ManualExtractionData/Android':
            df=pd.read_csv(path_name,skiprows=2,dtype='float')
            df.columns=[0,0,0,df.columns.values[3],df.columns.values[4],df.columns.values[5]]
            insert_line=pd.DataFrame(df.columns.values).T
            df.columns=insert_line.columns
            df=pd.concat([insert_line,df],axis=0)
        else:
            df=pd.read_csv(path_name,skiprows=2,dtype='float')
            insert_line=pd.DataFrame(df.columns.values).T
            df.columns=insert_line.columns
            df=pd.concat([insert_line,df],axis=0)
        df.columns=['Gx','Gy','Gz','Ax','Ay','Az']
        df['Ax']=df['Ax'].astype(float)
        df['Ay']=df['Ay'].astype(float)
        df['Az']=df['Az'].astype(float)
        df['Gx']=df['Gx'].astype(float)
        df['Gy']=df['Gy'].astype(float)
        df['Gz']=df['Gz'].astype(float)
        m1=df[['Ax','Ay','Az']]**2
        m2=df[['Gx','Gy','Gz']]**2
        df['AccSumAx']=np.array(m1.sum(axis=1))
        df['AccSumGx']=np.array(m2.sum(axis=1))
        col_name=list(df.columns.values)
        for c in col_name:
            df['fft'+'_'+c]=fft(df[c]).real
            df['ifft'+'_'+c]=ifft(df[c]).real
            df['idct'+'_'+c]=idct(df[c]).real
    except:
        df=pd.DataFrame(np.zeros(32)).T
        df.columns=['Gx', 'Gy', 'Gz', 'Ax', 'Ay', 'Az', 'AccSumAx', 'AccSumGx', 'fft_Gx',
       'ifft_Gx', 'idct_Gx', 'fft_Gy', 'ifft_Gy', 'idct_Gy', 'fft_Gz',
       'ifft_Gz', 'idct_Gz', 'fft_Ax', 'ifft_Ax', 'idct_Ax', 'fft_Ay',
       'ifft_Ay', 'idct_Ay', 'fft_Az', 'ifft_Az', 'idct_Az', 'fft_AccSumAx',
       'ifft_AccSumAx', 'idct_AccSumAx', 'fft_AccSumGx', 'ifft_AccSumGx',
       'idct_AccSumGx']
    return df

In [7]:
# you can generate the datasets while training models
# example
generate_read_table('AutomaticExtractionData_IMUZCenter','_Walk_1',104,label)

,Gx,Gy,Gz,Ax,Ay,Az,AccSumAx,AccSumGx,fft_Gx,ifft_Gx,...,idct_Ay,fft_Az,ifft_Az,idct_Az,fft_AccSumAx,ifft_AccSumAx,idct_AccSumAx,fft_AccSumGx,ifft_AccSumGx,idct_AccSumGx
0,-0.589232,-0.206970,-0.309588,-0.232,-1.196,-0.414,1.655636,0.485876,-14.851244,-0.027708,...,-682.405691,-34.816000,-0.064955,-38.502597,624.067340,1.164305,812.745359,189.541877,0.353623,251.176976
0,-0.610532,-0.105769,-0.330893,-0.134,-1.270,-0.430,1.815756,0.493427,-6.078164,-0.011340,...,219.930647,0.866556,0.001617,25.101857,-7.038980,-0.013132,-248.812970,-5.055658,-0.009432,-61.605677
1,-0.498682,-0.015222,-0.325567,-0.050,-1.330,-0.434,1.959756,0.354909,4.409632,0.008227,...,-139.915191,1.813574,0.003384,-7.946739,0.629383,0.001174,151.248365,-2.000619,-0.003732,16.881475
2,-0.370848,0.144568,-0.346872,0.006,-1.358,-0.422,2.022284,0.278748,1.127409,0.002103,...,92.249607,2.743805,0.005119,14.666210,3.352561,0.006255,-98.059829,4.733181,0.008831,-36.146651
3,-0.211058,0.155220,-0.219041,0.066,-1.360,-0.386,2.002952,0.116618,0.933910,0.001742,...,-82.503763,2.254026,0.004205,-3.532834,6.340125,0.011829,96.941503,17.113370,0.031928,7.429100
4,0.060584,-0.015222,-0.139146,0.052,-1.394,-0.270,2.018840,0.023264,-11.007014,-0.020535,...,56.986855,3.514593,0.006557,10.898762,6.288193,0.011732,-50.868789,15.797421,0.029473,-12.064023
5,-0.109858,-0.185664,-0.112514,0.058,-1.572,0.092,2.483012,0.059199,1.546707,0.002886,...,-60.160394,-0.230359,-0.000430,0.730624,1.396124,0.002605,73.130252,-5.860431,-0.010934,3.863630
6,-0.264321,0.123262,0.036623,0.230,-1.608,0.364,2.771060,0.086400,-4.564664,-0.008516,...,40.762830,1.438060,0.002683,8.569991,4.973296,0.009279,-31.435382,12.849838,0.023974,0.878087
7,-0.008658,0.352294,0.223044,0.496,-1.648,0.366,3.095876,0.173935,-10.450286,-0.019497,...,-49.038138,-1.360906,-0.002539,0.793218,0.803594,0.001499,71.018576,-4.274244,-0.007974,27.179754
8,-0.077900,0.474800,0.372181,0.516,-1.698,0.442,3.344824,0.370022,-9.560574,-0.017837,...,27.923921,-0.489438,-0.000913,7.233154,-10.819734,-0.020186,-14.062353,15.215019,0.028386,10.633294
